# <font color='blue'>Data Science Academy - Formação Cientista de Dados</font>
# <font color='blue'>Autor: Evandro Eulálio Cleto</font>

## <font color='blue'>Data Início: 07/06/2023</font>
## <font color='blue'>Data Finalização: 13/06/2023</font>


![title](imagens/Projeto_imagem.png)

## <font color='blue'>Objetivo deste projeto:</font>
### <font color='blue'>Através da análise de Tweets sobre o ChatGPT foi construído um processo de análise que permite identificar o sentimento que predomina, especialmente no Twitter, sobre o ChatGPT.</font>

Resumo do Projeto: Criar um projeto de previsão de sentimentos sobre ChatGPT atráves de Tweets on-line usando Machine Learning.
Os sentimentos serão previstos como positivo, negativo ou neutro.

Acesse http://localhost:4040 para acompanhar a execução dos jobs

## Spark Streaming - Twitter

In [ ]:
# Instalação de pacotes necessários para o projeto
#!pip install requests_oauthlib
#!pip install twython
#!pip install nltk
#!pip install emoji

In [ ]:
# https://pypi.org/project/findspark/
!pip install -q findspark

In [1]:
# Importa o findspark e inicializa
import findspark
findspark.init()

In [2]:
# Módulos usados
from pyspark.streaming import StreamingContext
#from pyspark.streaming.twitter import TwitterUtils
from pyspark import SparkContext
from pyspark.sql import SparkSession
from requests_oauthlib import OAuth1Session
from operator import add
import requests_oauthlib
from time import gmtime, strftime
import pandas as pd
import re
import requests
import time
import string
import ast
import json
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
# Pacote NLTK
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment import SentimentAnalyzer
from nltk.corpus import subjectivity
from nltk.corpus import stopwords
from nltk.sentiment.util import *

In [18]:
# Importa o arquivo CSV como DataFrame do pandas
# Esse é um dataset com 219294 registros de tweets do Chat GPT rotulado com sentimentos positivo, negativo que 
# será utilizado para treinamento do NaiveBayesClassifier.
# O dataset foi obtido em:  https://www.kaggle.com/datasets/charunisa/chatgpt-sentiment-analysis
df = pd.read_csv("dados/ChatGPT_sentiment_orig.csv",sep=",")

In [19]:
df.head(10)

,code,tweets,labels
0,0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,1,"Try talking with ChatGPT, our new AI system wh...",good
2,2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,3,"THRILLED to share that ChatGPT, our new model ...",good
4,4,"As of 2 minutes ago, @OpenAI released their ne...",bad
5,5,"Just launched ChatGPT, our new AI system which...",good
6,6,"As of 2 minutes ago, @OpenAI released their ne...",bad
7,7,ChatGPT coming out strong refusing to help me ...,good
8,8,#0penAl just deployed a thing I've been helpin...,good
9,9,Research preview of our newest model: ChatGPT\...,neutral


In [20]:
# Verificando o tipo do objeto
type(df)

pandas.core.frame.DataFrame

In [21]:
# Verificando o shape dos dados
df.shape

(219294, 3)

In [22]:
# Remove a coluna 'code' pois não tem relevância para o projeto
df = df.drop('code', axis=1)

In [23]:
df.head(10)

,tweets,labels
0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,"Try talking with ChatGPT, our new AI system wh...",good
2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,"THRILLED to share that ChatGPT, our new model ...",good
4,"As of 2 minutes ago, @OpenAI released their ne...",bad
5,"Just launched ChatGPT, our new AI system which...",good
6,"As of 2 minutes ago, @OpenAI released their ne...",bad
7,ChatGPT coming out strong refusing to help me ...,good
8,#0penAl just deployed a thing I've been helpin...,good
9,Research preview of our newest model: ChatGPT\...,neutral


In [24]:
# Altera posição da coluna 'labels' para a posição 0
cols = df.columns.tolist()
cols = ['labels'] + cols[:cols.index('labels')] + cols[cols.index('labels')+1:]
df = df[cols]

In [25]:
df.head(10)

,labels,tweets
0,neutral,ChatGPT: Optimizing Language Models for Dialog...
1,good,"Try talking with ChatGPT, our new AI system wh..."
2,neutral,ChatGPT: Optimizing Language Models for Dialog...
3,good,"THRILLED to share that ChatGPT, our new model ..."
4,bad,"As of 2 minutes ago, @OpenAI released their ne..."
5,good,"Just launched ChatGPT, our new AI system which..."
6,bad,"As of 2 minutes ago, @OpenAI released their ne..."
7,good,ChatGPT coming out strong refusing to help me ...
8,good,#0penAl just deployed a thing I've been helpin...
9,neutral,Research preview of our newest model: ChatGPT\...


In [26]:
# Remove vírgulas, exceto as do final da linha, da coluna 'tweets' evitar problemas na função que remove pontuação
df['tweets'] = df['tweets'].apply(lambda x: re.sub(r'(?<!\n),', '', x))

In [27]:
df.head(10)

,labels,tweets
0,neutral,ChatGPT: Optimizing Language Models for Dialog...
1,good,Try talking with ChatGPT our new AI system whi...
2,neutral,ChatGPT: Optimizing Language Models for Dialog...
3,good,THRILLED to share that ChatGPT our new model o...
4,bad,As of 2 minutes ago @OpenAI released their new...
5,good,Just launched ChatGPT our new AI system which ...
6,bad,As of 2 minutes ago @OpenAI released their new...
7,good,ChatGPT coming out strong refusing to help me ...
8,good,#0penAl just deployed a thing I've been helpin...
9,neutral,Research preview of our newest model: ChatGPT\...


In [28]:
# Mapear as classes para valores numéricos
label_mapping = {'bad': 0, 'good': 1, 'neutral': 2}
df['labels'] = df['labels'].map(label_mapping)

In [29]:
df.head(10)

,labels,tweets
0,2,ChatGPT: Optimizing Language Models for Dialog...
1,1,Try talking with ChatGPT our new AI system whi...
2,2,ChatGPT: Optimizing Language Models for Dialog...
3,1,THRILLED to share that ChatGPT our new model o...
4,0,As of 2 minutes ago @OpenAI released their new...
5,1,Just launched ChatGPT our new AI system which ...
6,0,As of 2 minutes ago @OpenAI released their new...
7,1,ChatGPT coming out strong refusing to help me ...
8,1,#0penAl just deployed a thing I've been helpin...
9,2,Research preview of our newest model: ChatGPT\...


In [30]:
# Remover emojis e caracteres especiais da coluna 'tweets'
df['tweets'] = df['tweets'].apply(lambda x: re.sub(r'[^\w\s,]', '', x))

In [31]:
df.head(10)

,labels,tweets
0,2,ChatGPT Optimizing Language Models for Dialogu...
1,1,Try talking with ChatGPT our new AI system whi...
2,2,ChatGPT Optimizing Language Models for Dialogu...
3,1,THRILLED to share that ChatGPT our new model o...
4,0,As of 2 minutes ago OpenAI released their new ...
5,1,Just launched ChatGPT our new AI system which ...
6,0,As of 2 minutes ago OpenAI released their new ...
7,1,ChatGPT coming out strong refusing to help me ...
8,1,0penAl just deployed a thing Ive been helping ...
9,2,Research preview of our newest model ChatGPTnn...


In [ ]:
# Obtém as stopwords em todos os idiomas
dicionario_stopwords = {lang: set(nltk.corpus.stopwords.words(lang)) for lang in nltk.corpus.stopwords.fileids()}

dicionario_stopwords

In [32]:
#Salva o datframe em csv
df.to_csv('dados/ChatGPT_sentiment_limpo.csv', index=False)

In [34]:
# Frequência de update
INTERVALO_BATCH = 5

In [35]:
# Cria o Spark Context
spark = SparkSession.builder.appName("TwitterSentimentAnalysis").getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

23/08/27 11:07:05 WARN Utils: Your hostname, DataScience resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/08/27 11:07:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/27 11:07:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [36]:
## Criando o StreamingContext
ssc = StreamingContext(sc, INTERVALO_BATCH)

## Treinando o Classificador de Análise de Sentimento

Uma parte essencial da criação de um algoritmo de análise de sentimento (ou qualquer algoritmo de mineração de dados) é ter um conjunto de dados abrangente ou "Corpus" para o aprendizado, bem como um conjunto de dados de teste para garantir que a precisão do seu algoritmo atende aos padrões que você espera. Isso também permitirá que você ajuste o seu algoritmo a fim de deduzir melhores (ou mais precisas) características de linguagem natural que você poderia extrair do texto e que vão contribuir para a classificação de sentimento, em vez de usar uma abordagem genérica. Tomaremos como base o dataset de treino fornecido pela Universidade de Michigan, para competições do Kaggle --> https://inclass.kaggle.com/c/si650winter11.

Esse dataset contém 1,578,627 tweets classificados e cada linha é marcada como: 

### 1 para o sentimento positivo 
### 0 para o sentimento negativo 

In [37]:
## Lendo o arquivo texto e criando um RDD em memória com Spark
arquivo = sc.textFile("dados/ChatGPT_sentiment_limpo.csv")

In [38]:
arquivo.collect()

['labels,tweets',
 '2,ChatGPT Optimizing Language Models for Dialogue httpstcoK9rKRygYyn OpenAI',
 '1,Try talking with ChatGPT our new AI system which is optimized for dialogue Your feedback will help us improve it httpstcosHDm57g3Kr',
 '2,ChatGPT Optimizing Language Models for Dialogue httpstcoGLEbMoKN6w AI MachineLearning DataScience ArtificialIntelligencennTrending AIML Article Identified amp Digested via Granola a MachineDriven RSS Bot by Ramsey Elbasheer httpstcoRprmAXUp34',
 '1,THRILLED to share that ChatGPT our new model optimized for dialog is now public free and accessible to everyone httpstcodyvtHecYbd httpstcoDdhzhqhCBX httpstcol8qTLure71',
 '0,As of 2 minutes ago OpenAI released their new ChatGPT nnAnd you can use it right now  httpstcoVyPGPNw988 httpstcocSn5h6h1M1',
 '1,Just launched ChatGPT our new AI system which is optimized for dialogue httpstcoArX6m0FfLEnnTry it out here httpstcoYM1gp5bA64',
 '0,As of 2 minutes ago OpenAI released their new ChatGPT nnAnd you can use i

In [39]:
##Removendo o cabeçalho
header = arquivo.take(1)[0]
dataset = arquivo.filter(lambda line: line!=header)

In [40]:
dataset.collect()

['2,ChatGPT Optimizing Language Models for Dialogue httpstcoK9rKRygYyn OpenAI',
 '1,Try talking with ChatGPT our new AI system which is optimized for dialogue Your feedback will help us improve it httpstcosHDm57g3Kr',
 '2,ChatGPT Optimizing Language Models for Dialogue httpstcoGLEbMoKN6w AI MachineLearning DataScience ArtificialIntelligencennTrending AIML Article Identified amp Digested via Granola a MachineDriven RSS Bot by Ramsey Elbasheer httpstcoRprmAXUp34',
 '1,THRILLED to share that ChatGPT our new model optimized for dialog is now public free and accessible to everyone httpstcodyvtHecYbd httpstcoDdhzhqhCBX httpstcol8qTLure71',
 '0,As of 2 minutes ago OpenAI released their new ChatGPT nnAnd you can use it right now  httpstcoVyPGPNw988 httpstcocSn5h6h1M1',
 '1,Just launched ChatGPT our new AI system which is optimized for dialogue httpstcoArX6m0FfLEnnTry it out here httpstcoYM1gp5bA64',
 '0,As of 2 minutes ago OpenAI released their new ChatGPT nnAnd you can use it right now n nhtt

In [41]:
type(dataset)

pyspark.rdd.PipelinedRDD

In [42]:
## Essa função separa as colunas em cada linha, cria uma tupla e remove a pontuação
def get_row(line):
    row = line.split(',')
    sentimento = row[0]
    tweet = row[1].strip()
    translator = str.maketrans({key: None for key in string.punctuation})
    tweet = tweet.translate(translator)
    tweet = tweet.split(' ')
    tweet_lower = []
    for word in tweet:
        tweet_lower.append(word.lower())
    return(tweet_lower,sentimento)

In [ ]:
#Aplica a função a cada linha do dataset
dataset_treino = dataset.map(lambda line: get_row(line))

In [ ]:
#Cria um objeto SentimentAnalyser
sentiment_analyser = SentimentAnalyzer()

In [ ]:
# Certifique-se de ter espaço em disco - Aproximadamente 5GB
# https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
# nltk.download()
nltk.download("stopwords")

In [ ]:
# Obtem a lista de stopwords
stopwords_all = []
for word in stopwords.words('english'):
    stopwords_all.append(word)
    stopwords_all.append(word + '_NEG')

In [ ]:
#Obtem 10.000 tweets do dataset de treino e retorna todas as palavras que não são Stpwords
dataset_treino_amostra = dataset_treino.take(10000)

In [ ]:
dataset_treino_amostra

In [ ]:
all_words_neg = sentiment_analyser.all_words([mark_negation(doc) for doc in dataset_treino_amostra])
all_words_neg_nostops = [x for x in all_words_neg if x not in stopwords_all]

In [ ]:
#Cria um unigram(n-grama) e extrai as features
unigram_feats = sentiment_analyser.unigram_word_feats(all_words_neg_nostops, top_n=200)
sentiment_analyser.add_feat_extractor(extract_unigram_feats, unigrams = unigram_feats)
training_set = sentiment_analyser.apply_features(dataset_treino_amostra)

In [ ]:
type(training_set)

In [ ]:
print(training_set)

In [ ]:
# Treinar o modelo
trainer = NaiveBayesClassifier.train
classifier = sentiment_analyser.train(trainer, training_set)

In [ ]:
# Testa o classificador em algumas sentenças
test_sentence1 = [(['model', 'is', 'people', 'bad'], '')]
test_sentence2 = [(['learning', 'day', 'bit', 'work', 'today'], '')]
test_sentence3 = [(['good', 'wonderful', 'results', 'awesome'], '')]
test_set = sentiment_analyser.apply_features(test_sentence1)
test_set2 = sentiment_analyser.apply_features(test_sentence2)
test_set3 = sentiment_analyser.apply_features(test_sentence3)

In [ ]:
test_set

In [ ]:
test_set2

In [ ]:
test_set3

In [ ]:
#Autenticação do Twitter
consumer_key = 'HZlZ9oKuUd9Pjy26EAPqW7P4a'
consumer_secret = 'JOcP5J0PmI7vbszwfw7ILJPtADA270l1UiuAOXeMJ5QLEJuu8n'
access_token = '1251925649952059392-8dKgUbCc0m9udPOlotSPzNC2UfJceJ'
access_token_secret ='2rhYbbD5OOlDRhzowz1OZza9LXPj6Gq0rpjPuOPLCMXbY'
bearer_token = "AAAAAAAAAAAAAAAAAAAAAOCeTwEAAAAAoL4M%2FzLMl%2FYAk3yCFrsc%2BOniGIM%3Dd8TzQTt3X4A1tyYmI2aCWEMRg4R3mYbarXPtiJBaA72xR7V2ev"

In [ ]:
# Configurar a autenticação do Twitter
auth_header = {
    "Authorization": "Bearer " + bearer_token,
    "Content-Type": "application/json"
}

In [ ]:
# Especifica a URL termo de busca
search_term = 'chatgpt'
sample_url ='https://stream.twitter.com/1.1/statuses/sample.json'
#filter_url = 'https://stream.twitter.com/1.1/statuses/filter.json?track='+search_term
#filter_url = 'https://api.twitter.com/2/tweets/search/stream?tweet.fields=text'+search_term
filter_url = "https://api.twitter.com/2/tweets/search/stream"
tweet_fields = "tweet.fields=text"

In [ ]:
query_params = {
    "expansions": "author_id",
    "tweet.fields": tweet_fields,
    "user.fields": "username",
    "track": search_term
}

In [ ]:
#Criando o objeto de autenticação para o Twitter
auth = requests_oauthlib.OAuth1(consumer_key, consumer_secret, access_token, access_token_secret)

In [ ]:
auth

In [ ]:
auth.

In [ ]:
auth = OAuth1Session('HZlZ9oKuUd9Pjy26EAPqW7P4a',
                            client_secret='JOcP5J0PmI7vbszwfw7ILJPtADA270l1UiuAOXeMJ5QLEJuu8n',
                            resource_owner_key='1251925649952059392-8dKgUbCc0m9udPOlotSPzNC2UfJceJ',
                            resource_owner_secret='2rhYbbD5OOlDRhzowz1OZza9LXPj6Gq0rpjPuOPLCMXbY')


In [ ]:
url = 'https://api.twitter.com/1/account/settings.json'

In [ ]:
r = auth.get(url)

In [ ]:
r

In [ ]:
#Configurando o Stream
rdd = ssc.sparkContext.parallelize([0])
stream = ssc.queueStream([], default=rdd)

In [ ]:
#Total de Tweets por update
NUM_TWEETS = 500

In [ ]:
type(stream)

In [ ]:
# Essa função conecta ao Twitter e retorna um número específico de Tweets (NUM_TWEETS)
def tfunc(t, rdd):
  return rdd.flatMap(lambda x: stream_twitter_data())

def stream_twitter_data():
   #response = requests.get(filter_url, auth = auth, stream = True)
  response = requests.get(filter_url, auth = auth, headers=auth_header, stream=True, params=query_params)
  print(filter_url, response)
  count = 0
  for line in response.iter_lines():
    try:
      if count > NUM_TWEETS:
        break
      post = json.loads(line.decode('utf-8'))
      contents = [post['text']]
      count += 1
      yield str(contents)
    except:
      result = False

In [ ]:
stream = stream.transform(tfunc)

In [ ]:
stream

In [ ]:
coord_stream = stream.map(lambda line: ast.literal_eval(line))

In [ ]:
# Essa função classifica os tweets, aplicando as features do modelo criado anteriormente
def classifica_tweet(tweet):
  sentence = [(tweet, '')]
  test_set = sentiment_analyzer.apply_features(sentence)
  print(tweet, classifier.classify(test_set[0][0]))
  return(tweet, classifier.classify(test_set[0][0]))

In [ ]:
# Essa função retorna o texto do Twitter
def get_tweet_text(rdd):
  for line in rdd:
    tweet = line.strip()
    translator = str.maketrans({key: None for key in string.punctuation})
    tweet = tweet.translate(translator)
    tweet = tweet.split(' ')
    tweet_lower = []
    for word in tweet:
      tweet_lower.append(word.lower())
    return(classifica_tweet(tweet_lower))

In [ ]:
# Cria uma lista vazia para os resultados
resultados = []

In [ ]:
# Essa função salva o resultado dos batches de Tweets junto com o timestamp
def output_rdd(rdd):
  global resultados
  pairs = rdd.map(lambda x: (get_tweet_text(x)[1],1))
  counts = pairs.reduceByKey(add)
  output = []
  for count in counts.collect():
    output.append(count)
  result = [time.strftime("%I:%M:%S"), output]
  resultados.append(result)
  print(result)

In [ ]:
# A função foreachRDD() aplica uma função a cada RDD to streaming de dados
coord_stream.foreachRDD(lambda t, rdd: output_rdd(rdd))

In [ ]:
# Start streaming
ssc.start()
# ssc.awaitTermination()

In [ ]:
cont = True
while cont:
  if len(resultados) > 5:
    cont = False

In [ ]:
# Grava os resultados
rdd_save = '/dados/r'+time.strftime("%I%M%S")
resultados_rdd = sc.parallelize(resultados)
resultados_rdd.saveAsTextFile(rdd_save)

In [ ]:
# Visualiza os resultados
resultados_rdd.collect()

In [ ]:
# Finaliza o streaming
ssc.stop()